In [62]:
import geopandas as gpd
import pandas as pd

In [63]:
# Load GeoJSON file
gdf = gpd.read_file("Parking_Violations_Issued_in_September_2024_original.geojson")

c:\Users\james\Anaconda3\envs\dev\lib\site-packages\geopandas\io\file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
c:\Users\james\Anaconda3\envs\dev\lib\site-packages\geopandas\io\file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
c:\Users\james\Anaconda3\envs\dev\lib\site-packages\geopandas\io\file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


In [64]:
gdf.head()

,OBJECTID,TICKET_NUMBER,VIOLATION_TYPE_DESC,ISSUE_DATE,ISSUE_TIME,ISSUING_AGENCY_CODE,ISSUING_AGENCY_NAME,ISSUING_AGENCY_SHORT,VIOLATION_CODE,VIOLATION_PROC_DESC,...,PENALTY_3,PENALTY_4,PENALTY_5,XCOORD,YCOORD,LATITUDE,LONGITUDE,MAR_ID,GIS_LAST_MOD_DTTM,geometry
0,83487425,260566401,P,2024-09-05 04:00:00+00:00,657,54,ST.ELZBETH HOSPITAL SECURITY GUARDS,SEH,P170,FAILURE TO DISPLAY CURRENT TAGS,...,0,0,0,NaN,NaN,NaN,NaN,NaN,2024-10-16 14:04:18+00:00,None
1,83487426,266969312,P,2024-09-18 04:00:00+00:00,133,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P055,NO PARKING ANYTIME,...,0,0,0,NaN,NaN,NaN,NaN,NaN,2024-10-16 14:04:18+00:00,None
2,83487445,266974724,P,2024-09-01 04:00:00+00:00,1020,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P012,DISOBEYING OFFICIAL SIGN,...,0,0,0,397076.780,135370.390,38.886,-77.034,810050.0,2024-10-16 14:04:18+00:00,None
3,83487446,266974735,P,2024-09-08 04:00:00+00:00,1051,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P170,FAILURE TO DISPLAY CURRENT TAGS,...,0,0,0,397328.708,135348.587,38.886,-77.031,805720.0,2024-10-16 14:04:18+00:00,None
4,83487447,266974746,P,2024-09-08 04:00:00+00:00,1108,59,US. BUREAU OF ENGRAVING AND PRINTNG,BEP,P170,FAILURE TO DISPLAY CURRENT TAGS,...,0,0,0,397328.637,135237.646,38.885,-77.031,801948.0,2024-10-16 14:04:18+00:00,None


In [65]:
gdf.columns

Index(['OBJECTID', 'TICKET_NUMBER', 'VIOLATION_TYPE_DESC', 'ISSUE_DATE',
       'ISSUE_TIME', 'ISSUING_AGENCY_CODE', 'ISSUING_AGENCY_NAME',
       'ISSUING_AGENCY_SHORT', 'VIOLATION_CODE', 'VIOLATION_PROC_DESC',
       'LOCATION', 'PLATE_STATE', 'VEHICLE_TYPE', 'MULTI_OWNER_NUMBER',
       'DISPOSITION_CODE', 'DISPOSITION_TYPE', 'DISPOSITION_DESC',
       'DISPOSITION_DATE', 'FINE_AMOUNT', 'TOTAL_PAID', 'PENALTY_1',
       'PENALTY_2', 'PENALTY_3', 'PENALTY_4', 'PENALTY_5', 'XCOORD', 'YCOORD',
       'LATITUDE', 'LONGITUDE', 'MAR_ID', 'GIS_LAST_MOD_DTTM', 'geometry'],
      dtype='object')

In [66]:
#Filter columns of interest
gdf_filtered = gdf[['ISSUE_DATE','ISSUE_TIME','ISSUING_AGENCY_NAME','VIOLATION_CODE','VIOLATION_PROC_DESC','LOCATION','FINE_AMOUNT','TOTAL_PAID','PENALTY_1','LATITUDE','LONGITUDE','geometry']].copy()

In [67]:
gdf_filtered.head()

,ISSUE_DATE,ISSUE_TIME,ISSUING_AGENCY_NAME,VIOLATION_CODE,VIOLATION_PROC_DESC,LOCATION,FINE_AMOUNT,TOTAL_PAID,PENALTY_1,LATITUDE,LONGITUDE,geometry
0,2024-09-05 04:00:00+00:00,657,ST.ELZBETH HOSPITAL SECURITY GUARDS,P170,FAILURE TO DISPLAY CURRENT TAGS,FRONT NEW HOSPITAL SE,100,0,100,NaN,NaN,None
1,2024-09-18 04:00:00+00:00,133,US. BUREAU OF ENGRAVING AND PRINTNG,P055,NO PARKING ANYTIME,ES D STREET SW,30,0,0,NaN,NaN,None
2,2024-09-01 04:00:00+00:00,1020,US. BUREAU OF ENGRAVING AND PRINTNG,P012,DISOBEYING OFFICIAL SIGN,BLK ES 300 RAOUL WALLENBERG PL SW,30,30,0,38.886,-77.034,None
3,2024-09-08 04:00:00+00:00,1051,US. BUREAU OF ENGRAVING AND PRINTNG,P170,FAILURE TO DISPLAY CURRENT TAGS,SS 1300 BLK C ST SW,100,0,0,38.886,-77.031,None
4,2024-09-08 04:00:00+00:00,1108,US. BUREAU OF ENGRAVING AND PRINTNG,P170,FAILURE TO DISPLAY CURRENT TAGS,NS 1300 BLK D ST SW,100,0,0,38.885,-77.031,None


In [68]:
import re
#COMBINE ISSUE_DATE AND ISSUE TIME INTO SINGLE COLUMN ISSUE_DATE

#Normalize time:
def normalizeTime(time):
    # Remove non-digit characters using regex
    time_str = re.sub(r'\D', '', str(time))  # \D matches anything that's not a digit
    # Zero-pad to ensure it's at least 4 characters long
    time_str = time_str.zfill(4)  
    hours = time_str[:-2]  # First two digits (hours)
    minutes = time_str[-2:]  # Last two digits (minutes)
    return f"{hours}:{minutes}" # HH:MM

#apply normalize function
gdf_filtered['ISSUE_TIME'] = gdf_filtered['ISSUE_TIME'].apply(normalizeTime)

#concatenate both strings and convert to datetime
gdf_filtered['ISSUE_DATE'] = pd.to_datetime(
    gdf_filtered['ISSUE_DATE'].dt.date.astype(str) + ' ' + gdf_filtered['ISSUE_TIME'], #date+ time ex: 2024-09-01 12:59
    format='%Y-%m-%d %H:%M'
)



In [69]:
gdf_filtered.head()

,ISSUE_DATE,ISSUE_TIME,ISSUING_AGENCY_NAME,VIOLATION_CODE,VIOLATION_PROC_DESC,LOCATION,FINE_AMOUNT,TOTAL_PAID,PENALTY_1,LATITUDE,LONGITUDE,geometry
0,2024-09-05 06:57:00,06:57,ST.ELZBETH HOSPITAL SECURITY GUARDS,P170,FAILURE TO DISPLAY CURRENT TAGS,FRONT NEW HOSPITAL SE,100,0,100,NaN,NaN,None
1,2024-09-18 01:33:00,01:33,US. BUREAU OF ENGRAVING AND PRINTNG,P055,NO PARKING ANYTIME,ES D STREET SW,30,0,0,NaN,NaN,None
2,2024-09-01 10:20:00,10:20,US. BUREAU OF ENGRAVING AND PRINTNG,P012,DISOBEYING OFFICIAL SIGN,BLK ES 300 RAOUL WALLENBERG PL SW,30,30,0,38.886,-77.034,None
3,2024-09-08 10:51:00,10:51,US. BUREAU OF ENGRAVING AND PRINTNG,P170,FAILURE TO DISPLAY CURRENT TAGS,SS 1300 BLK C ST SW,100,0,0,38.886,-77.031,None
4,2024-09-08 11:08:00,11:08,US. BUREAU OF ENGRAVING AND PRINTNG,P170,FAILURE TO DISPLAY CURRENT TAGS,NS 1300 BLK D ST SW,100,0,0,38.885,-77.031,None


In [70]:
# Remove rows with null LATITUDE values. Since LONGITUDE does not have additional nulls when LATITUDE is not null, this ensures both columns are cleaned.
gdf_filtered.dropna(subset=['LATITUDE'], inplace=True)

In [71]:
#confirm dropna worked
print(f'Latitude Nulls: {gdf_filtered["LATITUDE"].isnull().sum()}')
print(f'Longitude Nulls: {gdf_filtered["LONGITUDE"].isnull().sum()}')

Latitude Nulls: 0
Longitude Nulls: 0


In [72]:
#Check datatypes
gdf_filtered.dtypes

ISSUE_DATE             datetime64[ns]
ISSUE_TIME                     object
ISSUING_AGENCY_NAME            object
VIOLATION_CODE                 object
VIOLATION_PROC_DESC            object
LOCATION                       object
FINE_AMOUNT                     int64
TOTAL_PAID                      int64
PENALTY_1                       int64
LATITUDE                      float64
LONGITUDE                     float64
geometry                     geometry
dtype: object

In [73]:
# Drop the ISSUE_TIME column if it exists
gdf_filtered = gdf_filtered.drop(columns=['ISSUE_TIME'], errors='ignore')  # Ignore if column doesn't exist

#Rename columns
gdf_filtered.columns = ['date','agency_name','violation_code','violation_description','location','fine','paid','penalty','latitude','longitude','geometry']

In [74]:
#confirm column drop and rename
gdf_filtered.head()

,date,agency_name,violation_code,violation_description,location,fine,paid,penalty,latitude,longitude,geometry
2,2024-09-01 10:20:00,US. BUREAU OF ENGRAVING AND PRINTNG,P012,DISOBEYING OFFICIAL SIGN,BLK ES 300 RAOUL WALLENBERG PL SW,30,30,0,38.886,-77.034,None
3,2024-09-08 10:51:00,US. BUREAU OF ENGRAVING AND PRINTNG,P170,FAILURE TO DISPLAY CURRENT TAGS,SS 1300 BLK C ST SW,100,0,0,38.886,-77.031,None
4,2024-09-08 11:08:00,US. BUREAU OF ENGRAVING AND PRINTNG,P170,FAILURE TO DISPLAY CURRENT TAGS,NS 1300 BLK D ST SW,100,0,0,38.885,-77.031,None
5,2024-09-08 11:11:00,US. BUREAU OF ENGRAVING AND PRINTNG,P170,FAILURE TO DISPLAY CURRENT TAGS,NS 1300 BLK D ST SW,100,0,0,38.885,-77.031,None
6,2024-09-08 11:13:00,US. BUREAU OF ENGRAVING AND PRINTNG,P170,FAILURE TO DISPLAY CURRENT TAGS,SS 1300 BLK D ST SW,100,0,0,38.885,-77.031,None


In [75]:
# Check if gdf_filtered is a GeoDataFrame
is_geodataframe = isinstance(gdf_filtered, gpd.GeoDataFrame)
# IF GeoDataFrame: Export GeoDataFrame to GeoJSON
if(is_geodataframe):
    gdf_filtered.to_file("Cleaned_Parking_Violations_DC_09_2024.geojson", driver='GeoJSON')